# 使用omicverse[GPU]对scRNA-seq数据进行预处理

计数表是基因细胞的数字矩阵，是单细胞rna测序数据分析的基本输入数据结构。一个常见的预处理步骤是调整可变采样效率的计数，并对它们进行变换，使方差在整个动态范围内相似。

合适的预处理方法对scrna序列非常重要。在这里，我们介绍了一些预处理步骤，以帮助研究人员更容易地进行下游分析。

用户可以比较我们的教程和scanpy的教程来学习如何使用omicverse

Colab_Reproducibility：https://colab.research.google.com/drive/1DXLSls_ppgJmAaZTUvqazNC_E7EDCxUe?usp=sharing

## Installation

请注意，GPU模块不是直接存在的，需要单独安装，有关详细的教程请参阅https://rapids-singlecell.readthedocs.io/en/en/latest/index.html

```shell
pip
pip install rapids-singlecell
#rapids
pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12==24.4.* dask-cudf-cu12==24.4.* cuml-cu12==24.4.* \
    cugraph-cu12==24.4.* cuspatial-cu12==24.4.* cuproj-cu12==24.4.* \
    cuxfilter-cu12==24.4.* cucim-cu12==24.4.* pylibraft-cu12==24.4.* \
    raft-dask-cu12==24.4.* cuvs-cu12==24.4.*
#cupy
pip install cupy-cuda12x
```

### conda-env
请注意，为了避免冲突，我们将考虑在安装omicverse之前先安装快速单单元。

安装rapids-singlecell最简单的方法是使用conda文件夹中提供的一个yaml文件。这些yaml文件安装运行示例笔记本所需的所有内容并帮助您入门。

```shell
conda env create -f conda/omicverse_gpu.yml
# or
conda env create -f conda/omicverse_gpu.yml
```


In [2]:
import omicverse as ov
import scanpy as sc
ov.plot_set()
ov.settings.gpu_init()

All dependencies are satisfied.


TypeError: descriptor '__call__' for 'type' objects doesn't apply to a 'property' object

该数据由来自健康捐赠者的3k个pbmc组成，可从10x Genomics免费获得(此处来自此网页)。在unix系统上，您可以取消注释并运行以下命令来下载并解压缩数据。最后一行创建一个目录，用于写入处理过的数据。

In [1]:
# !mkdir data
#!wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz
#!cd data; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz
# !mkdir write

In [ ]:
adata = sc.read_10x_mtx(
    'data/filtered_gene_bc_matrices/hg19/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading
adata

## 预处理
### 质量控制

对于单细胞数据，我们需要在分析之前进行质量控制，包括去除包含双细胞，低表达细胞和低表达基因的细胞。除此之外，我们还需要根据线粒体基因比，成绩单数量，每个细胞表达的基因数，细胞复杂性等过滤。有关不同QC的详细说明，请参见文档：https：// hbcctraining。github.io/scrna-seq/lesson/04_sc_quality_control.html


In [ ]:
ov.pp.anndata_to_GPU(adata)



## 高变量基因检测
这里我们尝试使用皮尔逊的方法来计算高度可变的基因。这是建议优于普通归一化的方法。详见Nature Method中的文章。

`normalize| hvg`:我们使用`|`来控制预处理步骤，`|`之前用于归一化步骤，可以是`shiftlog`或`pearson`，之后用于高度可变的基因计算步骤，可以是`pearson`或`seurat`。我们的默认值是`shiftlog|pearson`。

- 如果你使用`mode=shiftlog|pearson`你需要设置`target_sum=50*1e4`，更多的人喜欢看到`target_sum=1e4`，更多人喜欢看到`target_sum=1e4`，我们测试的结果认为`50*1e4`会更好。
- 使用`mode=pearson|pearson`，你不需要设置`target_sum`。

> 如果' omicverse '的版本低于' 1.4.13 '，则模式只能在' scanpy '和' pearson '之间设置。

In [ ]:
adata=ov.pp.preprocess(adata,mode='shiftlog|pearson',n_HVGs=2000,)
adata

将AnnData对象的.raw属性设置为标准化和对数化的原始基因表达，以便稍后在差异测试和基因表达可视化中使用。这只是冻结了AnnData对象的状态。

In [ ]:
adata.raw = adata
adata = adata[:, adata.var.highly_variable_features]
adata

## 主成分分析
与scanpy相反，我们不直接缩放原始表达式矩阵的方差，而是将方差缩放的结果存储在层中，因为缩放可能会导致数据分布的变化，并且我们没有发现缩放在除主成分分析之外的任何场景中都是有意义的

In [ ]:
ov.pp.scale(adata)
adata

如果你想在`normlog`层中执行`pca`，你可以设置`layer=normlog`，但是我们认为在`pca`中缩放是必要的。